# Import

In [1]:
import yaml # pip install pyyaml 설치 필요
import wandb
import torch
import torch.nn as nn
import os
from tqdm import tqdm
from model.CNN import CNN
from utils.checkpoints import save_checkpoint, load_checkpoint

device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [2]:
device

device(type='cuda')

In [ ]:
# config 파일 로드
def loadConfig(configFile):
    with open(configFile, encoding='utf-8') as f:
        config = yaml.safe_load(f)
    return config

configFile = 'config.yaml'

# 아래처럼 사용
# learning_rate = config['learning_rate']
# epochs = config['epochs']
# batch_size = config['batch_size']
# loss_function = config['loss_function']


# Settings

# DataLoader

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset

In [4]:
BATCH_SIZE =  32 #한 배치당 32개 이미지데이터
EPOCHS = 40 # 전체 데이터 셋을 40번 반복
lr = 1e-2 # 학습률(learnign rate)

csv_file = ""
root_dir = "./data"

In [5]:
from torchvision import transforms
from utils.dataloader import CustomDataset

# 이미지 전처리 및 데이터셋 생성
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 원하는 크기로 리사이즈
    transforms.ToTensor(),           # 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

train_dataset = CustomDataset("", root_dir='./data', transform=transform)
val_dataset = CustomDataset("", root_dir='./data', transform=transform)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = True, # 순서가 암기되는것을 막기위해.
                                                    )
val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                                    batch_size = BATCH_SIZE,
                                                    shuffle = False, # 테스트 순서 유지.
                                                    ) 

In [6]:
len(train_dataset)

204

In [7]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

# Model Def

In [9]:
class AlexNet(nn.Module):
    def __init__(self, num_class=10):
        super(AlexNet, self).__init__()

        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=4),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 96, kernel_size=3),
            nn.ReLU(inplace=True)
        )
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc_layer1 = nn.Sequential(
            nn.Dropout(),
            nn.Linear(6400, 800),
            nn.ReLU(inplace=True),
            nn.Linear(800, 10)
        )

    def forward(self, x):
        output = self.conv_layer1(x)
        output = self.conv_layer2(output)
        output = self.conv_layer3(output)
        output = torch.flatten(output, 1)
        output = self.fc_layer1(output)
        return output


# Model Load & Loss Function

# TRAIN CODE

In [10]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [11]:
def binary_acc(y_pred, y_test):

    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    #checkpoint_filepath = 'checkpoint.pth'  # 체크포인트 저장 파일 경로
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [15]:
config = {'epoches': EPOCHS, 'batch_size': BATCH_SIZE, 'learning_rate': lr}
wandb.init(project='my-test-project', config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kihoon090. Use `wandb login --relogin` to force relogin


In [16]:
checkpoint_interval = 5 ## 5에폭마다 체크포인트 저장
best_val_loss = float('inf')  # 초기값 무한대

# checkpoints 폴더가 없으면 생성
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(EPOCHS):
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    ## Training
    print(f"Training on {epoch}")
    for images, labels in tqdm(train_loader, desc="train"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        pred = model(images)
        
        # pred => [0.2, 0.3, 0.5, 0] 
        # label => [0] or [1] or [2] or [3] 
        
        loss = criterion(pred, labels)
        
        # pred => argmax()
        # label  => [0] or [1] 
        pred = torch.argmax(pred, dim=1)
        acc = binary_acc(pred, labels)

        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += acc.item()
    
    train_loss = train_loss / len(train_dataset)
    train_acc = train_acc / len(train_dataset)
    print(f'Epoch {epoch+0:03}: | Loss: {train_loss:.5f} | Acc: {train_acc:.3f}')
    wandb.log({'accuracy': train_acc, 'loss': train_loss})
    
    ## Validation
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="val"):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)
        
            loss = criterion(pred, labels)
            pred = torch.argmax(pred, dim=1)
            acc = binary_acc(pred, labels)

            val_loss += loss.item()
            val_acc += acc.item()

    val_loss = val_loss / len(val_dataset)
    val_acc = val_acc / len(val_dataset)
    print(f'Epoch {epoch+0:03}: | Loss: {val_loss:.5f} | Acc: {val_acc:.3f}')
    wandb.log({'accuracy': val_acc, 'loss': val_loss})

    # validation이 개선되었을 때만 체크포인트 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint_filepath = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch + 1}.pth')  # 에포크 번호 포함
        save_checkpoint(model, optimizer, epoch, val_loss, checkpoint_filepath)
        print(f"Checkpoint updated at epoch {epoch + 1} and saved as {checkpoint_filepath}")

# 최종 체크포인트 저장
save_checkpoint(model, optimizer, epoch, train_loss / len(train_loader), 'final_checkpoint.pth')


Training on 0


train: 100%|██████████| 7/7 [00:03<00:00,  2.28it/s]


Epoch 000: | Loss: 0.04759 | Acc: 0.863


val: 100%|██████████| 7/7 [00:02<00:00,  2.62it/s]


Epoch 000: | Loss: 0.04763 | Acc: 0.784
Checkpoint saved at checkpoints\checkpoint_epoch_1.pth
Checkpoint updated at epoch 1 and saved as checkpoints\checkpoint_epoch_1.pth
Training on 1


...

Epoch 032: | Loss: 0.04751 | Acc: 0.784
Training on 33


train: 100%|██████████| 7/7 [00:02<00:00,  2.38it/s]


Epoch 033: | Loss: 0.04758 | Acc: 0.838


val:  29%|██▊       | 2/7 [00:01<00:03,  1.49it/s]


KeyboardInterrupt: 